In [106]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from scipy import spatial
p_raw = '/project/nsaru/raw/PySlake/'
p_work = '/project/nsaru/work/PySlake/'

In [107]:
def within(circle,point):
    if np.sqrt(np.dot(circle[0:2]-point,circle[0:2]-point))<circle[2]:
        return True
    else:
        return False

In [108]:
def nearest_cell(circles,centroids):
    cells_xy = circles[0][:,0:2]
    aggrs_xy = np.array(centroids)
    d = spatial.distance_matrix(cells_xy,aggrs_xy)
    cells_idx = []
    for id in range(d.shape[1]):
        i = np.argmin(d[:,id])
        if within(circles[0][i,:],aggrs_xy[id,:]):
            cells_idx.append(np.array([i,id]))
        
    return cells_idx

In [113]:
def cell_ids(circles,cell_labels):
    circles[0][:,0] = np.round(circles[0][:,0],decimals=-2)
    circles[0][:,1] = np.round(circles[0][:,1],decimals=-2)
    cdf = pd.DataFrame(circles[0][:,0:2], columns=['x', 'y'])
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    idx = np.array(cdf.index)
    return idx, cdf
    

In [111]:
samples = []
times = []
cells = []
all_matches = []
all_areas = []
all_circles = []
for d in os.listdir(p_raw):
    samples.append(d)
    t=[]
    matches = []
    areas = []
    circles_t = []
    for f in os.listdir(os.path.join(p_raw,d)):
        if '.csv' in f:
            if f!='ImageInfo.csv':
                cell_labels = pd.read_csv(os.path.join(p_raw,d,f))
    j = 0
    p_img = os.path.join(p_raw,d,'Images')
    if not os.path.isdir(p_img): 
        print(p_img)
        p_img = os.path.join(p_raw,d,'images')

    for f in os.listdir(p_img):
        t.append(os.path.getctime(os.path.join(p_img,f)))
        img = cv2.imread(os.path.join(p_img,f))
        b,g,r = cv2.split(img) 
        h,s,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
        if j==0:
            circles = cv2.HoughCircles(b,cv2.HOUGH_GRADIENT,1,175,param1=50,param2=30,minRadius=110,maxRadius=140)
            circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img,(i[0],i[1]),i[2],(255,0,0),5)

            #plt.subplot(211)
            #plt.imshow(img)
        ret, thresh = cv2.threshold(v,120,255,cv2.THRESH_BINARY_INV) 
        contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
        centers = []
        A = []
        for c in contours:
            # compute the center of the contour
            M = cv2.moments(c)
            if M["m00"]>0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                centers.append(np.array([cX,cY]))
                A.append(M["m00"])
        # draw contours on the original image
        cv2.drawContours(image=img, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=5, lineType=cv2.LINE_AA)

        #plt.subplot(212)
        #plt.imshow(img)
        #plt.show()
        if not os.path.isdir(os.path.join(p_work,d)): os.mkdir(os.path.join(p_work,d))
        fout = os.path.join(p_work,d,str(j)+'.jpg')
        cv2.imwrite(fout,img)
        cells_idx = nearest_cell(circles,centers)
        matches.append(cells_idx)
        areas.append(A)
        circles_t.append(circles)
        j = j+1
    all_circles.append(circles_t)
    all_matches.append(matches)
    all_areas.append(areas)
    times.append(t)

/project/nsaru/raw/PySlake/Kelly P24_Time1_Depth2/Images


In [114]:
cell_ids(circles,cell_labels)

(array([32, 18, 29, 17, 33, 34, 24, 16, 35, 27,  7, 22, 14,  4, 25, 15,  6,
        28, 26, 19,  3, 11,  1,  9,  2,  0, 20, 12, 10,  5,  8, 13, 21, 31,
        23, 30]),
        x     y
 32   550   500
 18   550  1400
 29   600  1000
 17   650   800
 33   650  1200
 34   650  1700
 24   700   500
 16   700  1000
 35   700  1400
 27   750  1700
 7    800   800
 22   800  1200
 14   850   500
 4    850  1000
 25   850  1400
 15   900   800
 6    900  1200
 28   900  1700
 26   950   500
 19   950  1000
 3    950  1400
 11  1050   800
 1   1050  1200
 9   1050  1700
 2   1100   500
 0   1100  1000
 20  1100  1400
 12  1150   800
 10  1150  1200
 5   1150  1700
 8   1250   500
 13  1250  1000
 21  1250  1400
 31  1300   800
 23  1300  1200
 30  1300  1700)